In [8]:
#load all desired packages
import sys
import pandas as pd
import numpy as np
#import random
#import time
#import torch

import os, sys
directory = os.path.abspath('')
sys.path.insert(1,os.path.join(directory,'src')) # setting path can also append directory.parent

#import utils
#import metrics
import datasets
import evaluate
#import plots
#import train
#from UNet import UNet

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
%load_ext autoreload
%autoreload 2
%matplotlib inline

ModuleNotFoundError: No module named 'scipy'

In [ ]:
#set training and validation batch sizes, other training parameters, and path to labels

# automatically updates params
code_path = '/data/knee_mri9/mwtong/t1rho_map_synthesis/Code'
log_save_dir   = code_path+'/train_logs/'
model_save_dir = code_path+'/checkpoints/'
model_info_file= '/data/knee_mri9/mwtong/t1rho_map_synthesis/training/trained_model_info.csv'

#Load the csv with all your trained models
df_modelInfo = pd.read_csv(model_info_file)

df_modelInfo

In [ ]:
ref_num = 1031
subset = 'test'
scaleMetrics_options = ['clip',0,100]
row_idx = np.where(df_modelInfo['Ref']==float(ref_num))[0][0]
preds,labels,images,testset = evaluate.get_model_preds(row_idx,df_modelInfo,subset)

# Get patient info so we can calculate metric per volume
row_idx = np.where(df_modelInfo['Ref']==float(ref_num))[0][0]
split_path = df_modelInfo['Split_Path'][row_idx]
label_df = pd.read_csv(split_path)
if subset == 'test':
    test_df = label_df.loc[label_df.set == 2, :]
elif subset == 'val':
    test_df = label_df.loc[label_df.set == 1, :]
elif subset == 'new_distribution':
    test_df = label_df.loc[label_df.set == -1, :]
elif subset == 'bilateral_knee':
    test_df = label_df.loc[label_df.set == -2, :]
    
df_patientInfo= evaluate.get_scan_info(test_df)
len(test_df)

In [ ]:
subset = 'bilateral_knee'
fname_slice_metrics = code_path+'/04_eval/'+subset+'_metrics/run'+str(ref_num)+'_slice_metrics.csv'
fname_vol_metrics = code_path+'/04_eval/'+subset+'_metrics/run'+str(ref_num)+'_vol_metrics.csv'
fname_vol_BA = code_path+'/04_eval/'+subset+'_metrics/run'+str(ref_num)+'_BA_vol_'+subset+'.png'
fname_vol_corr = code_path+'/04_eval/'+subset+'_metrics/run'+str(ref_num)+'_corr_vol_'+subset+'.png'
fname_slice_BA = code_path+'/04_eval/'+subset+'_metrics/run'+str(ref_num)+'_BA_slice_'+subset+'.png'
fname_slice_corr = code_path+'/04_eval/'+subset+'_metrics/run'+str(ref_num)+'_corr_slice_'+subset+'.png'

if (os.path.exists(fname_slice_metrics) and os.path.exists(fname_vol_metrics)):
    print('note: loading existing metric files')
    df_metrics_slice = pd.read_csv(fname_slice_metrics)
    df_metrics_slice['mean_pred']        =[list(eval(df_metrics_slice['mean_pred'][ii].replace(" ","")[1:-1])) for ii in range(len(df_metrics_slice))]
    df_metrics_slice['mean_targ']        =[list(eval(df_metrics_slice['mean_targ'][ii].replace(" ","")[1:-1])) for ii in range(len(df_metrics_slice))]
    df_metrics_slice['median_pred_targ'] =[list(eval(df_metrics_slice['median_pred_targ'][ii].replace(" ","")[1:-1])) for ii in range(len(df_metrics_slice))]

    df_metrics_vol = pd.read_csv(fname_vol_metrics)
    df_metrics_vol['mean_pred']        =[list(eval(df_metrics_vol['mean_pred'][ii].replace(" ","")[1:-1])) for ii in range(len(df_metrics_vol))]
    df_metrics_vol['mean_targ']        =[list(eval(df_metrics_vol['mean_targ'][ii].replace(" ","")[1:-1])) for ii in range(len(df_metrics_vol))]
    df_metrics_vol['median_pred_targ'] =[list(eval(df_metrics_vol['median_pred_targ'][ii].replace(" ","")[1:-1])) for ii in range(len(df_metrics_vol))]

else:
    df_metrics_slice,df_metrics_vol = evaluate.get_model_stats(preds,labels,images,scaleMetrics_options,df_patientInfo)

    # Save a summary of metrics in .../eval/run_metrics
    df_test_metric = pd.concat([test_df.reset_index(drop=False),df_metrics_slice],axis=1)
    df_test_metric.to_csv(code_path+'/04_eval/'+subset+'_metrics/run'+str(ref_num)+'_slice_metrics.csv')
    df_test_metric = pd.concat([df_patientInfo.reset_index(drop=False),df_metrics_vol],axis=1)
    df_test_metric.to_csv(code_path+'/04_eval/'+subset+'_metrics/run'+str(ref_num)+'_vol_metrics.csv')
